In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('tweets.csv', index_col=0)

In [4]:
df

,label,tweet,clear_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
id,,,,,,,
1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"['when', 'father', 'is', 'dysfunctional', 'and...","['father', 'dysfunctional', 'selfish', 'drags'...","['father', 'dysfunct', 'selfish', 'drag', 'kid...","['father', 'dysfunctional', 'selfish', 'drag',..."
2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use because they...,"['thanks', 'for', 'lyft', 'credit', 'can', 'no...","['thanks', 'lyft', 'credit', 'use', 'offer', '...","['thank', 'lyft', 'credit', 'use', 'offer', 'w...","['thank', 'lyft', 'credit', 'use', 'offer', 'w..."
3,0.0,bihday your majesty,bihday your majesty,"['bihday', 'your', 'majesty']","['bihday', 'majesty']","['bihday', 'majesti']","['bihday', 'majesty']"
4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in your,"['model', 'love', 'you', 'take', 'with', 'you'...","['model', 'love', 'take', 'time']","['model', 'love', 'take', 'time']","['model', 'love', 'take', 'time']"
5,0.0,factsguide: society now #motivation,factsguide society now motivation,"['factsguide', 'society', 'now', 'motivation']","['factsguide', 'society', 'motivation']","['factsguid', 'societi', 'motiv']","['factsguide', 'society', 'motivation']"
...,...,...,...,...,...,...,...
49155,NaN,thought factory: left-right polarisation! #tru...,thought factory left right polarisation trump ...,"['thought', 'factory', 'left', 'right', 'polar...","['thought', 'factory', 'left', 'right', 'polar...","['thought', 'factori', 'left', 'right', 'polar...","['think', 'factory', 'leave', 'right', 'polari..."
49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like mermaid hairflip neverready forma...,"['feeling', 'like', 'mermaid', 'hairflip', 'ne...","['feeling', 'like', 'mermaid', 'hairflip', 'ne...","['feel', 'like', 'mermaid', 'hairflip', 'never...","['feel', 'like', 'mermaid', 'hairflip', 'never..."
49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...,hillary campaigned today in ohio omg used word...,"['hillary', 'campaigned', 'today', 'in', 'ohio...","['hillary', 'campaigned', 'today', 'ohio', 'om...","['hillari', 'campaign', 'today', 'ohio', 'omg'...","['hillary', 'campaign', 'today', 'ohio', 'omg'..."


In [ ]:
#!pip install -U spacy
#!python -m spacy info

In [5]:
import spacy

In [7]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=393ce6abdd95733efc5ffa233d1b3838494e0e665c339cd90b32770706f39013
  Stored in directory: /tmp/pip-ephem-wheel-cache-5ougpbvz/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [8]:

from spacy import displacy
import en_core_web_md


In [ ]:
nlp = en_core_web_md.load()

In [ ]:
result=pd.DataFrame(columns=['NER', 'Name', 'Count'])
i=0
for i,values in df['clear_tweet'].iteritems():
  doc=nlp(str(values))
  for ent in doc.ents:
    result.loc[i,'NER']=ent.label_
    result.loc[i,'Name']=ent.text
    result.loc[i,'Count']=1
    i+=1
  

In [ ]:
result

,NER,Name,Count
2,ORG,lyft credit,1
3,ORG,bihday,1
5,ORG,factsguide society,1
7,DATE,tomorrow,1
8,DATE,the next school year,1
...,...,...,...
49156,PERSON,mermaid hairflip neverready,1
49157,PERSON,hillary,1
49158,DATE,today,1
49159,PERSON,newmusic newsong,1


In [ ]:
top=result.loc[:,['NER','Count']].groupby('NER').sum()

In [ ]:
top.sort_values(by='Count', ascending =False)

,Count
NER,
PERSON,13616
DATE,8224
ORG,7193
GPE,4046
TIME,1725
NORP,1388
CARDINAL,909
ORDINAL,425
FAC,251


In [ ]:
person_spicy=result[result['NER']=='PERSON'].groupby('Name').sum().sort_values(by='Count', ascending =False)

In [ ]:
person_spicy['Count'].head(20)

Name
bihday                                                     140
bing bong bing bong                                        107
lighttherapy                                                97
obama                                                       83
yoyou                                                       64
hu                                                          50
hillary                                                     43
feminismiscancer feminismisterrorism feminismmuktbharat     40
essentialoils                                               39
staed                                                       36
fathersday                                                  34
sea shepherd suppoers                                       28
brexit blm                                                  27
christina grimmie                                           26
shi                                                         26
lebron                                            

Спайси отработал скорее плохо, чем хорошо. Из топ 20 персон действительно людьми оказались лишь 6. 

In [ ]:
person_org=result[result['NER']=='ORG'].groupby('Name').sum().sort_values(by='Count', ascending =False)

In [ ]:
person_org['Count'].head(20)

Name
fathersday                 251
bihday                     128
allahsoil                  104
sjw                        101
gop                         48
instagram                   40
nba                         39
google                      30
stas                        27
cavs                        27
tgif ff                     26
brexit                      25
udtapunjab                  25
gif                         25
update social analytics     24
disney                      23
islam                       22
unfounately                 22
impoant                     21
eur usd                     19
Name: Count, dtype: int64

Тоже касается и организаций. Из ТОП-20 лишь 5-6 являются организациями, много мусора.

In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [72]:
i=0
for sent in nltk.sent_tokenize(sentence):
  ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(sent)))
  named_entities = []
  for tagged_tree in ne_tree:
    print(tagged_tree)
    if hasattr(tagged_tree, 'label'):
      entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #
      entity_type = tagged_tree.label() # get NE category
      nltk_df.loc[i,'NER']=entity_type
      nltk_df.loc[i,'Name']=entity_name
      named_entities.append((entity_name, entity_type))
      i+=1
  print(named_entities)
  #print(cs)
  #print(sent)  

('thought', 'JJ')
('factory', 'NN')
('left', 'VBD')
('right', 'JJ')
('polarisation', 'NN')
('trump', 'NN')
(PERSON Donald/NNP)
('google', 'NN')
(PERSON Google/NNP)
('.', '.')
[('Donald', 'PERSON'), ('Google', 'PERSON')]
('i', 'NN')
('want', 'VBP')
('bmw', 'NN')
[]


In [ ]:
nltk_df=pd.DataFrame(columns=['NER', 'Name', 'Count'])

In [101]:
i=0
for j, sentence in df['tweet'].iteritems():
  for sent in nltk.sent_tokenize(str(sentence)):
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
        if hasattr(chunk, 'label'):
          #print(chunk.label(), ' '.join(c[0] for c in chunk))
          nltk_df.loc[i,'NER']=chunk.label()
          nltk_df.loc[i,'Name']=' '.join(c[0] for c in chunk)
          nltk_df.loc[i,'Count']=1
          i+=1
              

In [102]:
nltk_df

,NER,Name,Count
0,GPE,en-ger-land,1
1,GPE,en-ger-land,1
2,GPE,brisk_and_vagabond-eyeopener_2007__enviromenta...,1
3,GPE,u.s.,1
4,ORGANIZATION,nycÂ,1
...,...,...,...
111,ORGANIZATION,vysoÄina,1
112,ORGANIZATION,euro2016Â,1
113,GPE,brisk_and_ham-taste_the_rainbow__rock_da_pay-,1
114,ORGANIZATION,braggingÂ,1


NLTK очень плохо отработал. Почти все NER оказались мусором. На очищенных твитах он вообще ничего не искал, только на не предобработанных (заглавные буквы для него важны). Пробовал делать тест, писал в предложении Google и google, и google он не определял, как сущность. Сомнительное выделение NER.
